# Swin-T Tiny Training for RSNA Cervical Spine Fracture Detection

# Goal: Train 5-fold Swin-T tiny (seed=789) for diversity in 4-way ensemble with ConvNeXt v1/v2 + RegNet.
# Adapt from 02_baseline_cnn.ipynb: same MIP data, augs, BCEWithLogitsLoss, pos_weight, sampler, HFlip TTA.
# Model: timm.create_model('swin_tiny_patch4_window7_224', num_classes=7, in_chans=3, drop_rate=0.3, drop_path_rate=0.1).
# Resolution: 224x224 (Swin native), bone-windowed MIPs resized.
# Save: fold_{}_swin.pth, oof_logits_swin_tta.npy.
# After training, blend as 4th model with capped weights (Swin <=0.4), re-calibrate temps/LR/alpha, target CV<0.40 silver.
# Estimated time: ~2h on A10 GPU (5 folds, 20 epochs each with early stopping).

# Next: Load data/folds, define model/dataset, train loop, generate OOF TTA logits, compute CV WLL, request expert review.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import log_loss
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import os
import random
import subprocess
from scipy.special import expit as sigmoid
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

# GPU check from best practices
print(subprocess.run(['bash', '-lc', 'nvidia-smi || true'], capture_output=True, text=True).stdout)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set seed for reproducibility
SEED = 789
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(SEED)

# Load data (train_mips.csv already includes labels, no merge needed)
df = pd.read_csv('data/train_mips.csv')
print(f'df shape: {df.shape}, columns: {df.columns.tolist()}')
label_cols = ['C1','C2','C3','C4','C5','C6','C7']
y = df[label_cols].fillna(0).values  # Fill NaN with 0 if any
groups = df['StudyInstanceUID'].values
print(f'y shape: {y.shape}, groups shape: {groups.shape}, NaN in y: {np.isnan(y).sum()}')

# Folds (GroupKFold n_splits=5, random_state=42)
gkf = GroupKFold(n_splits=5)
folds = []
for fold, (tr_idx, va_idx) in enumerate(gkf.split(df, y, groups=groups)):
    folds.append((tr_idx, va_idx))
print(f'Loaded {len(df)} samples, 5 folds ready.')

# Model definition
def build_swin_model():
    model = timm.create_model('swin_tiny_patch4_window7_224',
                              pretrained=False, num_classes=7, in_chans=3,
                              drop_rate=0.3, drop_path_rate=0.1)
    model = model.to(device)
    return model

# Dataset (adapt to 224x224)
class MIPDataset(Dataset):
    def __init__(self, df, labels, transform=None):
        self.df = df.reset_index(drop=True)
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        uid = self.df.iloc[idx]['StudyInstanceUID']
        mip_path = f'data/mips/train/{uid}.npy'
        if not os.path.exists(mip_path):
            print(f'Missing MIP for {uid}')
            # For subsample, assume all exist or skip
            mip = np.zeros((3, 224, 224), dtype=np.float32)  # fallback zero
        else:
            mip = np.load(mip_path).astype(np.float32)  # (3,H,W)
        img = np.transpose(mip, (1,2,0))  # HWC
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return img, label

# Transforms (224x224, same augs as ConvNeXt)
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])

# Loss and sampler (BCEWithLogitsLoss with pos_weight, WeightedRandomSampler)
pos_weight = torch.tensor([2.0] * 7).to(device)  # adjust if needed
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

def get_sampler(df, labels):
    class_counts = np.sum(labels, axis=0)
    n_samples = len(df)
    weights = 1.0 / (class_counts + 1e-6)
    sample_weights = np.mean([weights[c] for c in range(7) if labels[:,c].sum() > 0], axis=0) * np.ones(n_samples)
    return WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

print('Setup complete: Swin-T model, 224 res transforms, data/folds ready. Next: implement training loop for 5 folds, save weights/OOF TTA logits, compute CV WLL.')

Fri Sep 26 08:26:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |   14924MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Training loop: 5 folds, Swin-T, BCEWithLogitsLoss, AdamW lr=1e-4, early stopping on val WLL, HFlip TTA for OOF
import time
from sklearn.metrics import log_loss
def wll(y_true, p_pred):
    p_pred = np.clip(p_pred, 1e-6, 1-1e-6)
    losses = [log_loss(y_true[:,i], p_pred[:,i], labels=[0,1]) for i in range(7)]
    return np.average(losses, weights=[1]*7)

N_FOLDS = 5
BATCH_SIZE = 16  # Smaller for Swin-T
EPOCHS = 20
LR = 1e-4
PATIENCE = 5
oof_logits = np.zeros((len(df), 7), dtype=np.float32)
fold_scores = []
for fold in range(N_FOLDS):
    tr_idx, va_idx = folds[fold]
    print(f'\n=== Fold {fold+1}/{N_FOLDS} ===')
    tr_df = df.iloc[tr_idx].reset_index(drop=True)
    va_df = df.iloc[va_idx].reset_index(drop=True)
    tr_labels = y[tr_idx]
    va_labels = y[va_idx]
    
    tr_ds = MIPDataset(tr_df, tr_labels, train_transform)
    va_ds = MIPDataset(va_df, va_labels, val_transform)
    
    sampler = get_sampler(tr_df, tr_labels)
    tr_dl = DataLoader(tr_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=True)
    va_dl = DataLoader(va_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    
    model = build_swin_model()
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    best_val_loss = float('inf')
    patience_counter = 0
    fold_start = time.time()
    
    for epoch in range(EPOCHS):
        epoch_start = time.time()
        model.train()
        tr_loss = 0.0
        for imgs, lbls in tqdm(tr_dl, desc=f'Epoch {epoch+1} Train'):
            imgs, lbls = imgs.to(device), lbls.to(device)
            optimizer.zero_grad()
            logits = model(imgs)
            loss = criterion(logits, lbls)
            loss.backward()
            optimizer.step()
            tr_loss += loss.item()
        tr_loss /= len(tr_dl)
        
        model.eval()
        va_logits = []
        va_loss = 0.0
        with torch.no_grad():
            for imgs, lbls in tqdm(va_dl, desc=f'Epoch {epoch+1} Val'):
                imgs, lbls = imgs.to(device), lbls.to(device)
                logits = model(imgs)
                loss = criterion(logits, lbls)
                va_loss += loss.item()
                va_logits.append(logits.cpu().numpy())
        va_loss /= len(va_dl)
        va_logits = np.concatenate(va_logits, axis=0)
        p_va = sigmoid(va_logits)
        val_wll = wll(va_labels, p_va)
        
        scheduler.step(val_wll)
        
        elapsed = time.time() - epoch_start
        print(f'Epoch {epoch+1}: tr_loss={tr_loss:.4f}, val_loss={va_loss:.4f}, val_wll={val_wll:.4f}, lr={optimizer.param_groups[0]["lr"]:.2e}, time={elapsed:.1f}s')
        
        if val_wll < best_val_loss:
            best_val_loss = val_wll
            patience_counter = 0
            torch.save(model.state_dict(), f'fold_{fold+1}_swin.pth')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    fold_time = time.time() - fold_start
    print(f'Fold {fold+1} done: best_val_wll={best_val_loss:.4f}, time={fold_time/60:.1f}min')
    fold_scores.append(best_val_loss)
    
    # OOF TTA logits (HFlip)
    model.load_state_dict(torch.load(f'fold_{fold+1}_swin.pth', map_location='cpu', weights_only=True))
    model.to(device).eval()
    fold_oof_logits = []
    with torch.no_grad():
        for imgs, _ in va_dl:
            imgs = imgs.to(device)
            logits = model(imgs)
            logits_f = model(torch.flip(imgs, dims=[3]))
            tta_logits = 0.5 * (logits + logits_f)
            fold_oof_logits.append(tta_logits.cpu().numpy())
    oof_logits[va_idx] = np.concatenate(fold_oof_logits, axis=0)
    
# Compute CV WLL
oof_p = sigmoid(oof_logits)
cv_wll = wll(y, oof_p)
print(f'\nSwin-T CV WLL: {cv_wll:.4f} (mean fold: {np.mean(fold_scores):.4f} \u00b1 {np.std(fold_scores):.4f})')
np.save('oof_logits_swin_tta.npy', oof_logits)
print('Saved oof_logits_swin_tta.npy. Next: ensemble with 3-way, re-optimize weights/LR/alpha, target <0.40 WLL, request expert review.')



=== Fold 1/5 ===


Epoch 1 Train:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 1 Train:   9%|▉         | 1/11 [00:00<00:02,  3.63it/s]

Epoch 1 Train:  27%|██▋       | 3/11 [00:00<00:01,  7.23it/s]

Epoch 1 Train:  45%|████▌     | 5/11 [00:00<00:00,  8.84it/s]

...

Epoch 14 Val:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 14 Val:  50%|█████     | 1/2 [00:00<00:00,  3.72it/s]

Epoch 14 Val: 100%|██████████| 2/2 [00:00<00:00,  6.09it/s]

Epoch 14: tr_loss=0.4993, val_loss=0.6044, val_wll=0.3428, lr=5.00e-05, time=1.5s
Early stopping at epoch 14
Fold 5 done: best_val_wll=0.3384, time=0.4min



Swin-T CV WLL: 0.3463 (mean fold: 0.3462 ± 0.0397)
Saved oof_logits_swin_tta.npy. Next: ensemble with 3-way, re-optimize weights/LR/alpha, target <0.40 WLL, request expert review.
